In [ ]:
from datetime import datetime, timedelta
from dask.distributed import LocalCluster

from hrrr_db.db.models import Site 
from hrrr_db.hrrr import HRRR, HRRRGrid, VariableInfo

# References
https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/python_data_loading.html

In [ ]:
cluster = LocalCluster(n_workers=6, threads_per_worker=1, memory_limit='4GB')
client = cluster.get_client()
client

## Dates of interest

In [ ]:
october_1st = datetime.fromisoformat('2023-10-01T00:00:00')
date_list = [october_1st + timedelta(days=x) for x in range(10)]

## Mores Creek

In [ ]:
MCS_record = Site.load('Mores Creek Summit')

In [ ]:
mcs_HRRR = HRRR(True)
mcs_variable = VariableInfo(
    run_hour=date_list[0], 
    level='surface', 
    name='TMP'
)

# HRRR AWS

### Single point

In [ ]:
mcs_HRRR.set_chunk_info_for_lon_lat(MCS_record.geom.x, MCS_record.geom.y)

### Single value

In [ ]:
data = mcs_HRRR.load_tile(mcs_variable, forecast_hour=6)
data

### Whole grid chunk containing the point

In [ ]:
whole_grid = mcs_HRRR.load_dataset(mcs_variable)

In [ ]:
x, y = HRRRGrid.transform_lon_lat(
    MCS_record.geom.x, MCS_record.geom.y
)

### Grab the point from the chunk

In [ ]:
mcs_point = whole_grid.sel(
    x=x, y=y, method="nearest"
)

mcs_point

In [ ]:
mcs_point.TMP.values[5]

# Local File (slow)

Requires to manually download the desired day from AWS or Google

In [ ]:
import xarray as xr

In [ ]:
grib_f = xr.open_mfdataset(
    '/path/to/file.grib',
    filter_by_keys={
        'stepType': 'instant', 
        'typeOfLevel': 'surface'
    },
    parallel=True
)
grib_f = grib_f.drop_vars(['time'])

In [ ]:
grib_f = grib_f.metpy.assign_crs(HRRRGrid.CRS.to_cf())
grib_f = grib_f.metpy.assign_y_x()
grib_f = grib_f.set_coords("valid_time")

In [ ]:
x, y = HRRRGrid.transform_lon_lat(
     MCS_record.geom.x, MCS_record.geom.y
)

mcs_grib_f = grib_f.sel(
    x=x, y=y, method="nearest"
)
mcs_grib_f

In [ ]:
mcs_grib_f.t.values

In [ ]:
client.shutdown()